# 🏦 Bank Customer Churn Prediction
## Notebook 7 — Production Inference Module

**Goal:** Demonstrate the `CustomerChurn` class from `BankChurn_Module.py` — a clean, reusable Python module that encapsulates the entire inference pipeline.

### Why package the inference pipeline into a class?

When deploying a machine learning model in the real world (e.g., as an API, a scheduled job, or an embedded tool), you need a consistent, reproducible way to:
1. Load the model and scaler.
2. Accept raw input data.
3. Apply the *exact same* preprocessing as during training.
4. Return predictions in a human-readable format.

Putting all of this in a well-documented class makes the code:
- **Reusable** — import and use from any script or application.
- **Maintainable** — one place to update if preprocessing logic changes.
- **Testable** — each method can be independently unit-tested.

In [ ]:
# Import the module we built (must be in the same directory or on the Python path)
from BankChurn_Module import CustomerChurn

print('BankChurn_Module imported successfully ✓')

## 1. Module Architecture Overview

```
BankChurn_Module.py
│
├── class CustomScaler
│     ├── __init__()    — store columns list + StandardScaler
│     ├── fit()         — learn mean/std from training data
│     └── transform()   — apply scaling, preserve column order
│
└── class CustomerChurn
      ├── __init__()              — load model.pkl + scaler.pkl
      ├── load_and_clean_data()   — read CSV → drop cols → scale → encode → reindex
      └── predict_churn()         — run model → attach predictions → return DataFrame
```

The design follows the **Single Responsibility Principle** — each method does exactly one thing.

## 2. Instantiate the Model

Creating a `CustomerChurn` object loads both the model and the scaler into memory.  
This is done once and the object is then reused for all predictions.

In [ ]:
# Instantiate — loads model_file.pkl and Scaler_file.pkl
churn_predictor = CustomerChurn(
    model_file  = 'model_file.pkl',
    scaler_file = 'Scaler_file.pkl'
)

print('Model type   :', type(churn_predictor.model_selected))
print('Scaler type  :', type(churn_predictor.scaler_selected))
print('Ready for inference ✓')

## 3. Load and Preprocess New Data

`load_and_clean_data()` mirrors the training pipeline steps:
1. Read raw CSV.
2. Drop identifier/leakage columns.
3. Scale numerical features using the **pre-fitted** scaler (transform only — no re-fitting).
4. One-hot encode categorical features.
5. Reindex to the model's expected feature order.

In [ ]:
# For this demo we use the original CSV (which contains 'Exited').
# In production, the input CSV would be new customers WITHOUT the 'Exited' column.
# The module will silently skip dropping 'Exited' if it's absent (errors='ignore').

preprocessed_data = churn_predictor.load_and_clean_data('Customer-Churn-Records.csv')

print(f'Preprocessed features shape: {preprocessed_data.shape}')
print(f'Expected columns: {churn_predictor.FEATURE_COLUMNS}')
print()
preprocessed_data.head()

## 4. Generate Predictions

In [ ]:
results = churn_predictor.predict_churn()

print(f'Output DataFrame shape: {results.shape}')
print(f'Columns: {results.columns.tolist()}')
print()
# Show a human-readable view: key identifiers + prediction
display_cols = ['CustomerId', 'Surname', 'Geography', 'Gender', 'Age',
                'Balance', 'NumOfProducts', 'IsActiveMember', 'Predicted_Exited']
# Only include columns that exist in results
display_cols = [c for c in display_cols if c in results.columns]
results[display_cols].head(10)

In [ ]:
# Prediction distribution
pred_counts = results['Predicted_Exited'].value_counts()
print('Prediction distribution:')
print(f'  Predicted to STAY   : {pred_counts.get(0, 0):,}  ({pred_counts.get(0, 0)/len(results)*100:.1f}%)')
print(f'  Predicted to CHURN  : {pred_counts.get(1, 0):,}  ({pred_counts.get(1, 0)/len(results)*100:.1f}%)')

## 5. ⚠️ In-Sample Check (Not True Validation)

Since our demo CSV includes the actual `Exited` column, we can run a sanity check.

**Important caveat:** The final model in N6 was trained on ALL 10,000 rows of this same file. Comparing its predictions to the actual labels here is **in-sample evaluation** — we are measuring how well the model memorised its own training data, not how well it generalises.

**True generalisation performance** was measured in N5 on the held-out 20% test set (real, unseen, imbalanced data). Those are the metrics to report.

This cell is included only as a deployment sanity check: if accuracy here is not near 100%, something is broken in the pipeline.

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

if 'Exited' in results.columns:
    acc = accuracy_score(results['Exited'], results['Predicted_Exited'])
    print(f'In-sample accuracy (training data — NOT a generalisation metric): {acc:.4f}')
    print('  → See N5 test-set metrics for true generalisation performance.')
    print()
    print(classification_report(results['Exited'], results['Predicted_Exited'],
                                 target_names=['Stayed', 'Churned']))
else:
    print('No actual labels available for comparison (production mode).')

## 6. Export Predictions

The output DataFrame can be saved to a CSV for downstream use — uploading to a CRM, scheduling retention campaigns, or further analysis.

In [ ]:
output_path = 'churn_predictions_output.csv'
results.to_csv(output_path, index=False)
print(f'✅ Predictions saved to  {output_path}')
print(f'   Rows: {len(results):,}  |  Columns: {len(results.columns)}')

---
## ✅ Project Complete — End-to-End Summary

| Notebook | Task | Key Output |
|---|---|---|
| N1 | Data upload & first look | Dataset shape, dtypes, data dictionary |
| N2 | EDA | Distributions, correlations, leakage finding |
| N3 | Data cleaning | `df_cleaned.csv` (14 cols, 10K rows) |
| N4 | Feature engineering | `data_processed.csv`, `Scaler_file.pkl` |
| N5 | Model training & selection | Random Forest selected |
| N6 | Final model saving | `model_file.pkl` |
| **N7** | **Inference module** | **`churn_predictions_output.csv`** |

### Possible Next Steps
- **Hyperparameter tuning** — `GridSearchCV` on `n_estimators`, `max_depth`, `min_samples_leaf`.
- **SHAP values** — Explain individual predictions ("why did customer X get flagged?").
- **Flask/FastAPI deployment** — Wrap `CustomerChurn` class in an HTTP endpoint.
- **Monitoring** — Track prediction drift over time as new customer data arrives.